FIle untuk latihan

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
!pip install Sastrawi
from Sastrawi.Dictionary.ArrayDictionary import ArrayDictionary
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.1 MB/s eta 0:00:00


##Prepare Dataset

In [ ]:
# Membuat DataFrame kosong
df = pd.DataFrame()

# Membaca file CSV ke dalam DataFrame df_test
df_test = pd.read_csv('/content/train_preprocess.tsv.txt')


#https://drive.google.com/drive/folders/1-1J5AtUsKdeBGPtAmuLd2kmIHSn22VE3

# Menggabungkan df_test ke dalam DataFrame df
df = df.append(df_test, ignore_index=True)

<ipython-input-8-467f58cbbf2b>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_test, ignore_index=True)


In [ ]:
df

,id,text,label
0,411,"Dekat dengan hotel saya menginap, hanya ditemp...",positive
1,729,"Iya benar, dia sedang jaga warung.",neutral
2,373,Kangkungnya lumayan tapi kepiting saus padangn...,negative
3,262,Bertempat di braga city walk yang satu gedung ...,positive
4,177,Gianyar terima bantuan sosial 2018 sebesar rp ...,neutral
...,...,...,...
395,502,Saya sudah sering kali datang menikmati makana...,positive
396,268,Banyak orang yang kurang suka untuk berobat ke...,negative
397,282,"Pelayanan baik, tempat parkir cukup luas, kebu...",positive
398,407,"Demi apa pun tes cpns bikin macet, sialannnnnnn",negative


In [ ]:
df.head()

,id,text,label
0,411,"Dekat dengan hotel saya menginap, hanya ditemp...",positive
1,729,"Iya benar, dia sedang jaga warung.",neutral
2,373,Kangkungnya lumayan tapi kepiting saus padangn...,negative
3,262,Bertempat di braga city walk yang satu gedung ...,positive
4,177,Gianyar terima bantuan sosial 2018 sebesar rp ...,neutral


In [ ]:
df.shape

(400, 3)

In [ ]:
df.label.value_counts()

negative    153
positive    151
neutral      96
Name: label, dtype: int64

In [ ]:
df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
395    False
396    False
397    False
398    False
399    False
Length: 400, dtype: bool

##Text Normalization

In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
def word(cleaned_text):
    return word_tokenize(cleaned_text)
df['text_token'] = df['text'].apply(word)
df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,id,text,label,text_token
0,411,"Dekat dengan hotel saya menginap, hanya ditemp...",positive,"[Dekat, dengan, hotel, saya, menginap, ,, hany..."
1,729,"Iya benar, dia sedang jaga warung.",neutral,"[Iya, benar, ,, dia, sedang, jaga, warung, .]"
2,373,Kangkungnya lumayan tapi kepiting saus padangn...,negative,"[Kangkungnya, lumayan, tapi, kepiting, saus, p..."
3,262,Bertempat di braga city walk yang satu gedung ...,positive,"[Bertempat, di, braga, city, walk, yang, satu,..."
4,177,Gianyar terima bantuan sosial 2018 sebesar rp ...,neutral,"[Gianyar, terima, bantuan, sosial, 2018, sebes..."
...,...,...,...,...
395,502,Saya sudah sering kali datang menikmati makana...,positive,"[Saya, sudah, sering, kali, datang, menikmati,..."
396,268,Banyak orang yang kurang suka untuk berobat ke...,negative,"[Banyak, orang, yang, kurang, suka, untuk, ber..."
397,282,"Pelayanan baik, tempat parkir cukup luas, kebu...",positive,"[Pelayanan, baik, ,, tempat, parkir, cukup, lu..."
398,407,"Demi apa pun tes cpns bikin macet, sialannnnnnn",negative,"[Demi, apa, pun, tes, cpns, bikin, macet, ,, s..."


In [ ]:
#menghilangkat kata-kata yang tidak terlalu di perlukan
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Unduh kamus kata-kata stop words Bahasa Indonesia
nltk.download('stopwords')
nltk.download('punkt')  # Unduh data tokenisasi

# Inisialisasi set stop words
wordlist = set(stopwords.words('indonesian'))

# Membuat set kata-kata tambahan
additional_stopwords = set([
    "yg", "dg", "rt", "dgn", "ny", "d", 'klo', 'kalo', 'amp',
    'biar', 'bikin', 'bilang', 'gak', 'ga', 'krn', 'nya', 'nih',
    'sih', 'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 'jd', 'jgn',
    'sdh', 'aja', 'n', 't', 'nyg', 'hehe', 'pen', 'u', 'nan',
    'loh', 'user', 'bukan', 'hanya', 'kata', 'gua', '&amp', 'yah'
    , 'orang', 'lu', 'url', 'gue', 'tp', 'gw', 'udah', 'lo', 'url'
    , 'dah', 'jg', 'org', 'emang', 'pake'
    , 'pa','ni','mah','iya','bgt','tu','gk','liat'
])

# Gabungkan set stop words NLTK dan set kata-kata tambahan
list_stopwords = wordlist.union(additional_stopwords)

# Buat DataFrame df sesuai dengan kebutuhan Anda

# Membuat fungsi untuk menghapus stop words
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

# Menggunakan word_tokenize untuk membagi teks menjadi token
tokenizer = word_tokenize
df['text_token'] = df['text'].apply(lambda text: tokenizer(text))

# Menghapus stop words dan kata-kata tambahan
df['text_remove_stopwords'] = df['text_token'].apply(stopwords_removal)

# Menggabungkan token-token menjadi satu string
df['text_remove_stopwords'] = df['text_remove_stopwords'].apply(lambda tokens: ' '.join(tokens))

# Tampilkan DataFrame yang telah diproses
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,id,text,label,text_token,text_remove_stopwords
0,411,"Dekat dengan hotel saya menginap, hanya ditemp...",positive,"[Dekat, dengan, hotel, saya, menginap, ,, hany...","Dekat hotel menginap , ditempuh jalan kaki , p..."
1,729,"Iya benar, dia sedang jaga warung.",neutral,"[Iya, benar, ,, dia, sedang, jaga, warung, .]","Iya , jaga warung ."
2,373,Kangkungnya lumayan tapi kepiting saus padangn...,negative,"[Kangkungnya, lumayan, tapi, kepiting, saus, p...",Kangkungnya lumayan kepiting saus padangnya me...
3,262,Bertempat di braga city walk yang satu gedung ...,positive,"[Bertempat, di, braga, city, walk, yang, satu,...",Bertempat braga city walk gedung aston fave ho...
4,177,Gianyar terima bantuan sosial 2018 sebesar rp ...,neutral,"[Gianyar, terima, bantuan, sosial, 2018, sebes...","Gianyar terima bantuan sosial 2018 rp 44 , 9 m..."
...,...,...,...,...,...
395,502,Saya sudah sering kali datang menikmati makana...,positive,"[Saya, sudah, sering, kali, datang, menikmati,...",Saya kali menikmati makanan roemah nenek . Say...
396,268,Banyak orang yang kurang suka untuk berobat ke...,negative,"[Banyak, orang, yang, kurang, suka, untuk, ber...",Banyak suka berobat dokter . Selain biaya rela...
397,282,"Pelayanan baik, tempat parkir cukup luas, kebu...",positive,"[Pelayanan, baik, ,, tempat, parkir, cukup, lu...","Pelayanan , parkir luas , kebun stroberi luas ..."
398,407,"Demi apa pun tes cpns bikin macet, sialannnnnnn",negative,"[Demi, apa, pun, tes, cpns, bikin, macet, ,, s...","Demi tes cpns macet , sialannnnnnn"


#Feature Extraction

##Bag of Words

In [ ]:
data_preprocessed = df.text.tolist()
data_preprocessed

['Dekat dengan hotel saya menginap, hanya ditempuh jalan kaki, di sini banyak sekali pilihan makanannya, tempat yang luas, dan menyenangkan',
 'Iya benar, dia sedang jaga warung.',
 'Kangkungnya lumayan tapi kepiting saus padangnya mengecewakan kami dikasih kepiting yang kopong akhir kami tidak makan keptingnya dan dikembalikan.',
 'Bertempat di braga city walk yang satu gedung dengan aston dan fave hotel, tempat ini sangat nyaman buat kongkow-kongkow. Kopi campur teh yang baru pertama kali saya nikmati ternyata sangat enak, dipadu dengan telur setengah matang menjadi pendamping mengobrol bersama teman-teman. Area yang bebas merokok semakin mengasyikkan sambil menikmati pemandangan lalu lalang orang-orang yang keluar masuk mal ini.',
 'Gianyar terima bantuan sosial 2018 sebesar rp 44, 9 miliar',
 'Ini salah satu contoh generasi yang tidak berguna. Bukan malah bermanfaat untuk bangsa tapi malah jadi sesuatu yang berbahaya bagi bangsa! Mantap jerat saja pakai uu ite',
 'Sore sahabat, saa

In [ ]:
# Proses Feature Extraction
count_vect = CountVectorizer()

X = count_vect.fit_transform(data_preprocessed)
print("Feature Extraction selesai")
print(X)

Feature Extraction selesai
  (0, 483)	1
  (0, 486)	1
  (0, 809)	1
  (0, 1914)	1
  (0, 1392)	1
  (0, 778)	1
  (0, 628)	1
  (0, 870)	1
  (0, 929)	1
  (0, 497)	1
  (0, 2081)	1
  (0, 186)	1
  (0, 1965)	1
  (0, 1739)	1
  (0, 1231)	1
  (0, 2204)	1
  (0, 2412)	1
  (0, 1206)	1
  (0, 466)	1
  (0, 1433)	1
  (1, 861)	1
  (1, 232)	1
  (1, 498)	1
  (1, 1939)	1
  (1, 867)	1
  :	:
  (398, 447)	1
  (398, 2059)	1
  (399, 1914)	2
  (399, 1231)	2
  (399, 2284)	1
  (399, 1871)	1
  (399, 860)	1
  (399, 1889)	1
  (399, 475)	1
  (399, 1599)	2
  (399, 1134)	1
  (399, 1418)	1
  (399, 1629)	1
  (399, 175)	1
  (399, 1723)	1
  (399, 1791)	1
  (399, 206)	1
  (399, 1612)	1
  (399, 2384)	1
  (399, 711)	1
  (399, 1748)	1
  (399, 670)	1
  (399, 748)	1
  (399, 962)	1
  (399, 2019)	1


In [ ]:
count_array = X.toarray()
df = pd.DataFrame(data=count_array,columns = count_vect.get_feature_names_out())
print(df)

     00  000  10  100  101  11  12  120  13  14  ...  yang  yin  yogi  \
0     0    0   0    0    0   0   0    0   0   0  ...     1    0     0   
1     0    0   0    0    0   0   0    0   0   0  ...     0    0     0   
2     0    0   0    0    0   0   0    0   0   0  ...     1    0     0   
3     0    0   0    0    0   0   0    0   0   0  ...     4    0     0   
4     0    0   0    0    0   0   0    0   0   0  ...     0    0     0   
..   ..  ...  ..  ...  ...  ..  ..  ...  ..  ..  ...   ...  ...   ...   
395   0    0   0    0    0   0   0    0   0   0  ...     3    0     0   
396   0    0   0    0    0   0   0    0   0   0  ...     3    0     0   
397   0    0   0    0    0   0   0    0   0   0  ...     3    0     0   
398   0    0   0    0    0   0   0    0   0   0  ...     0    0     0   
399   0    0   0    0    0   0   0    0   0   0  ...     0    0     0   

     yogyakarta  yooyee  youtube  yummi  yun  yuummy  zaman  
0             0       0        0      0    0       0      0  

#TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Proses Feature Extraction
tfidf_vect = TfidfVectorizer()

X = tfidf_vect.fit_transform(data_preprocessed)
print("Feature Extraction selesai")
print(X)

Feature Extraction selesai
  (0, 1433)	0.3009071853661943
  (0, 466)	0.11549650946206552
  (0, 1206)	0.26584691279857803
  (0, 2412)	0.10290288829167932
  (0, 2204)	0.1630905778888982
  (0, 1231)	0.21180434291254835
  (0, 1739)	0.22318251343555495
  (0, 1965)	0.21835953165604288
  (0, 186)	0.18566049603599427
  (0, 2081)	0.1926378930256465
  (0, 497)	0.11062664754525402
  (0, 929)	0.3009071853661943
  (0, 870)	0.21835953165604288
  (0, 628)	0.3410003937474862
  (0, 778)	0.2312666618481542
  (0, 1392)	0.2919907596751321
  (0, 1914)	0.1331884010691585
  (0, 809)	0.2312666618481542
  (0, 486)	0.14362825895329234
  (0, 483)	0.2774541619295394
  (1, 2397)	0.3945342311566633
  (1, 867)	0.4732745199821702
  (1, 1939)	0.3440699660981449
  (1, 498)	0.36896898757442087
  (1, 232)	0.3766204223809986
  :	:
  (398, 105)	0.27612434518818313
  (398, 320)	0.2801476442840195
  (399, 2019)	0.251386827000835
  (399, 962)	0.251386827000835
  (399, 748)	0.251386827000835
  (399, 670)	0.251386827000835
  (3

In [ ]:
tfidf_array = X.toarray()
df = pd.DataFrame(data=tfidf_array,columns = tfidf_vect.get_feature_names_out())
print(df)

      00  000   10  100  101   11   12  120   13   14  ...      yang  yin  \
0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.102903  0.0   
1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000  0.0   
2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.077035  0.0   
3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.177138  0.0   
4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000  0.0   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...       ...  ...   
395  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.190948  0.0   
396  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.141732  0.0   
397  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.152060  0.0   
398  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000  0.0   
399  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000  0.0   

     yogi  yogyakarta  yooyee  youtube  yummi  yun  yuummy  zaman  
0     0

In [ ]:
import numpy as np
import math
math.log(4/1)

1.3862943611198906

In [ ]:
# Proses Feature Extraction
count_vect = TfidfVectorizer()

X = count_vect.fit_transform(data_preprocessed)
print("Feature Extraction selesai")

Feature Extraction selesai
